In [1]:
import preprocessor_end
import os
from shutil import copy
import pickle
import pandas as pd

In [2]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/data_dictionary_before_removing_frequent_words.pkl','rb')
data_dictionary = pickle.load(pickle_in)


In [3]:
pandas_dict = {}

pandas_dict["rating"] = []
pandas_dict ["movie_id"] = []
pandas_dict ["movie_words"] = []



In [4]:
for label in list(data_dictionary.keys())[:2]:
    for movie_name in data_dictionary[label]:
        pandas_dict["rating"].append(label)
        pandas_dict["movie_words"].append(data_dictionary[label][movie_name])
        pandas_dict["movie_id"].append(movie_name)

In [5]:
data_frames = pd.DataFrame(data=pandas_dict, columns=["rating","movie_id","movie_words"])

In [3]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/pandas_data_frame.pkl','rb')
data_frames = pickle.load(pickle_in)

In [6]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/word2vec_model_trained_150.pkl','rb')
model = pickle.load(pickle_in)

C:\Users\Abdo\PycharmProjects\final_project_1\venv\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
max = (0, "movie_name")
min = (100000000,"movie_name")
list_of_lengths_and_id = []

for index, row in data_frames.iterrows():
    movie_id = row["movie_id"]
    rating = row["rating"]
    length = len(row["movie_words"])
    list_of_lengths_and_id.append( (length, movie_id, rating) )

In [8]:
list_of_lengths_and_id.sort()

In [9]:
# find the max, and min number of words
min_number_of_words = list_of_lengths_and_id[0]
max_number_of_words = list_of_lengths_and_id[-1]

In [10]:
# find the average number of words
total = 0
for file in list_of_lengths_and_id:
    total = total + file[0]
average_number_of_words = total / len(list_of_lengths_and_id)

In [13]:
vocabulary = set(model.wv.vocab)

In [29]:
dict_of_1_label_words = {}
dict_of_0_label_words = {}

In [30]:
# dict_of_0_label_words: {word: , word:, ......}
# word:
#   [number_of_total_occurrence, {movie_name:number_of_occurrence_in_movie, movie_name:number_of_occurrence_in_movie, .....}]
for movie_name in data_dictionary["0_label"]:
    for word in data_dictionary["0_label"][movie_name]:
        if word in dict_of_0_label_words:
            dict_of_0_label_words[word][0] = dict_of_0_label_words[word][0] + 1
            if(movie_name in dict_of_0_label_words[word][1]):
                (dict_of_0_label_words[word][1])[movie_name] = (dict_of_0_label_words[word][1])[movie_name] + 1
            else:
                (dict_of_0_label_words[word][1])[movie_name] = 1
        else:
            dict_of_0_label_words[word] = [1, {movie_name:1}]

In [31]:
for movie_name in data_dictionary["1_label"]:
    for word in data_dictionary["1_label"][movie_name]:
        if word in dict_of_1_label_words:
            dict_of_1_label_words[word][0] = dict_of_1_label_words[word][0] + 1
            if(movie_name in dict_of_1_label_words[word][1]):
                (dict_of_1_label_words[word][1])[movie_name] = (dict_of_1_label_words[word][1])[movie_name] + 1
            else:
                (dict_of_1_label_words[word][1])[movie_name] = 1
        else:
            dict_of_1_label_words[word] = [1, {movie_name:1}]

In [38]:
#with open('C:/Users/Abdo/Project/pickle_files/dict_of_1_label_words.pkl', 'wb') as output:
    #pickle.dump(dict_of_1_label_words, output)

In [39]:
#with open('C:/Users/Abdo/Project/pickle_files/dict_of_0_label_words.pkl', 'wb') as output:
    #pickle.dump(dict_of_0_label_words, output)

In [14]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/dict_of_1_label_words.pkl','rb')
dict_of_1_label_words = pickle.load(pickle_in)

In [15]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/dict_of_0_label_words.pkl','rb')
dict_of_0_label_words = pickle.load(pickle_in)

In [70]:
print(dict_of_0_label_words["terrorist"])

[23, {'PG_1010': 2, 'PG_124': 1, 'PG_139': 1, 'PG_1474': 2, 'PG_1483': 1, 'PG_1822': 1, 'PG_2046': 1, 'PG_2162': 1, 'PG_281': 1, 'PG_3137': 1, 'PG_3240': 1, 'PG_3602': 2, 'PG_3842': 1, 'PG_3996': 1, 'PG_4740': 2, 'PG_5092': 1, 'PG_6151': 1, 'PG_6269': 1, 'PG_6652': 1}]


In [44]:
#number_of_total_words_in_all_movies
all_text_words = set(dict_of_1_label_words.keys())
all_text_words.update(set(dict_of_0_label_words.keys()))

In [48]:
common_words = set(dict_of_1_label_words.keys()).intersection(set(dict_of_0_label_words.keys()))

In [49]:
percentage_of_common_words = len(common_words)/len(all_text_words)

In [53]:
#check for non-English words
data_frames

,rating,movie_id,movie_words
0,0_label,GP_1500,"[good, morning, sir, damn, strange, business, ..."
1,0_label,GP_2470,"[sergeant, martin, yes, detective, inspector, ..."
2,0_label,GP_3199,"[download, movie, subtitles, load, directly, u..."
3,0_label,GP_3581,"[chattering, man, bugle, plays, want, remember..."
4,0_label,GP_4816,"[morning, pancho, good, morning, child, need, ..."
5,0_label,GP_5893,"[miss, eady, christie, come, blessed, blackout..."
6,0_label,GP_6068,"[restored, digitized, support, technical, mean..."
7,0_label,G_1014,"[well, pismo, beach, clams, eat, put, right, b..."
8,0_label,G_1025,"[tanya, fievel, stop, twirling, twirling, uh, ..."
9,0_label,G_108,"[thudding, sizzling, dish, washer, running, ah..."
